In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import pandas as pd
from datasets import load_dataset
import faiss
import numpy as np


In [4]:
retriever_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the retriever model
retriever_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Load data from the CSV file
df = pd.read_csv("151_ideas_updated.csv")  # Replace with the correct file path

# Ensure the correct column is used for passages
df.columns = df.columns.str.strip()  # Remove any extra spaces in column names
passages = df["Ideas"].dropna().tolist()  # Replace "Ideas" with the actual column name if different

# Encode the passages
passage_embeddings = retriever_model.encode(passages, convert_to_tensor=True)

# Print the shape of the embeddings
print("Encoded passages:", passage_embeddings.shape)


Encoded passages: torch.Size([150, 768])


In [6]:
import faiss
import numpy as np

# Ensure the embeddings are converted to a NumPy array
passage_embeddings_np = passage_embeddings.cpu().numpy()  # Convert tensor to NumPy array

# Create a FAISS index
index = faiss.IndexFlatL2(passage_embeddings_np.shape[1])  # Dimensionality of the embeddings

# Add embeddings to the index
index.add(passage_embeddings_np)

print("Number of embeddings in index:", index.ntotal)


Number of embeddings in index: 150


In [7]:
generator_tokenizer = AutoTokenizer.from_pretrained("t5-small")
generator_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


In [14]:
def retrieve_and_generate(query, top_k=5, min_tokens=500, max_length=1500):
    # Encode the query
    query_embedding = retriever_model.encode([query], convert_to_tensor=True).cpu().numpy()

    # Retrieve top_k passages
    _, indices = index.search(query_embedding, top_k)
    retrieved_passages = [passages[i] for i in indices[0]]

    # Combine retrieved passages
    input_text = f"{query} {' '.join(retrieved_passages)} Discuss failures, beauty, and the certainty of death with examples."

    # Generate response
    input_ids = generator_tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = generator_model.generate(
        input_ids,
        max_length=max_length,
        min_length=min_tokens,
        temperature=0.7,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=2,
        early_stopping=False,
        length_penalty=1.2  # Encourage longer responses
    )

    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
query = "list points that mention beauty"
print(retrieve_and_generate(query, min_tokens=500, max_length=1000))


/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:657: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.2` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


a feminist, but not enough to be feminist. I see half the world suffering from cynicism that is stunting civilization. gitty up! Update 2-19-19 Feminism is important to me because I've seen half of the population suffering... but if that’s not sufficient to have feminism, I don’t know what is. Prejudice is inherently dumb. If you see something say something, right? We see its extreme importance in all of our lives. We saw it».'....................)??.............................................? fa  ne -', just., 'MtB – slut..,.“””..: Patriarchy?...? (.t:..!...and d'' ::)? and! e.g. when i, are &?; ;. (M.W.B. more than one, and have the p.- t,.&. in mtb, this ’m:-;&& o. and how l. — and for...........–...no,, it,...to... and even, at yo...,........... h.s. the. [ed.;]:(...) the beauty.â») as tru.o;; or, we see:?!.» () in every.’: [)-. but. that. just r.a. or w. on this:,;)..C.self.T.A..com;-;...the.p;, as. all. people.i.P.E. of."B" v..S.M:E in our hearts.R.• and to. ".>. »" in: "D.f.m.